In [2]:
import numpy as np
from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import *
from qiskit.algorithms.optimizers import GradientDescent
from qiskit.quantum_info.operators import Operator
from qiskit.primitives import Estimator
from qiskit.circuit import Parameter
from qiskit.quantum_info import DensityMatrix


## Hamiltonian

In [3]:
def Hamiltonian(N, a):
    def krZ(m, N):
        matrix = np.array(1)
        for j in range(N):
            if m == j:
                matrix = np.kron(matrix, ZGate())     
            else:
                matrix = np.kron(matrix, IGate())
        return matrix  

    def krId(N):
        matrix = np.array(1)
        for j in range(N):
            matrix = np.kron(matrix, IGate())
        return matrix  

    def doublesum(N):
        sum = 0
        for i in range(N):
            for j in range(N):
                sum += 2**(i+j) * np.matmul(((krId(N)-krZ(i,N))/2.0),((krId(N)-krZ(j,N))/2.0))
        return sum

    def sum(N):
        sum = 0
        for i in range(N):
            sum += 2**(i) * ((krId(N)-krZ(i, N))/2.0)

        return sum
        
    return a**2 *(0.25*krId(N)+sum(N)+ doublesum(N))

## Ansatz

In [4]:
def Ansatz(depth, N, param_num):
    if param_num > N**2:
        raise Exception('number of parameters are bigger then the rotation gates possible')
        
    circuit = QuantumCircuit(N)
    thetas = []
    for l in range(param_num):
        thetas.append(Parameter('θ_'+str(l)))
    
    for i in range(depth):
        circuit.barrier()
        counter = 0
        
        for j in range(N):
            if j+counter <len(thetas):
                circuit.rx(thetas[j+counter],j)
            if j+1+counter <len(thetas):
                circuit.ry(thetas[j+1+counter],j)
            if j+2+counter< len(thetas):
                circuit.rz(thetas[j+2+counter],j)
            else: 
                break
            counter += 2
            
        for p in range(N):
            if p+1 < len(range(N)):
                circuit.cx(p,(p+1))
            else:
                break
    return circuit

## Cost Function

In [33]:
def costfn(param):
    state = Ansatz(1, 4, 9)
    binded = state.bind_parameters({state.parameters[i]:param[i] for i in range(len(param))})
    op = Operator(Hamiltonian(4, 0.5))
    estimator = Estimator()
    expectation_value = estimator.run(binded, op).result().values[0]  
    return expectation_value

## Find optimized state

In [27]:
opt = ADAM()

def optimization(theta_start,cost_fun):
    optimized = opt.minimize(fun=cost_fun, x0=theta_start)
    state = Ansatz(1,4,9)
    statevector = state.bind_parameters({state.parameters[i]:optimized.x[i] for i in range(len(optimized.x))})
    density = DensityMatrix(statevector)
    return optimized.x, statevector, density
    
    

In [1]:
optstate = optimization(np.zeros(9), costfn)[1]

NameError: name 'optimization' is not defined

In [34]:
optimized2=optimization(np.zeros(9), costfn)

KeyboardInterrupt: 

In [39]:
optimized2=opt.minimize(fun=costfn, x0=np.zeros(9))

In [41]:
print(optimized2)

{   'fun': 0.06344750975465213,
    'jac': None,
    'nfev': 1000,
    'nit': 100,
    'njev': 0,
    'x': array([-5.00000000e-03, -5.00000000e-03, -3.86912724e-14, -4.99647242e-03,
       -4.99647242e-03, -2.15105711e-14, -3.89671401e-03, -3.89671401e-03,
       -2.03725925e-14])}


In [22]:
def krId(N):
        matrix = np.array(1)
        for j in range(N):
            matrix = np.kron(matrix, IGate())
        return matrix 

In [5]:
Hamiltonian(2,1)

array([[ 0.25,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  6.25,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  2.25,  0.  ],
       [ 0.  ,  0.  ,  0.  , 12.25]])